In [1]:
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger

# tfがエラーはかないため
#import tensorflow as tf
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


# 課題1

* Wikipediaの適当な英語のエントリをダウンロードする。

 * 選ぶのが面倒という方はAIのエントリでもどうぞ。
   * https://en.wikipedia.org/wiki/Artificial_intelligence

* BeautifulSoupで本文のテキストだけを取得する。

 * HTMLのソースを見て、どこが本文かを確認する。
 * あるいは、ネット検索をして、Wikipediaのエントリから本文だけを取得する方法を調べる。

* 以下の前処理をする。

 * 大文字は小文字にする。ただし固有名詞を除く。

 * ストップワードを除去する。

 * lemmatizationする。

* 各単語の出現回数を求め、表示する。

* lemmatizationした後の単語を、元の出現順序どおりに、半角スペースで区切ってつなぎ、長い一つの文字列にする。
 * joinメソッドを使えばよい。つまり、__`' '.join(`__ lemmatizeされた単語のリスト __`)`__ という感じ。

In [2]:
import requests
import re
from bs4 import BeautifulSoup
data = requests.get('https://en.wikipedia.org/wiki/Yoshi')
content = data.text

In [25]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    contents = soup.find('div', id="content")
#        soup.find('div', id="id_name")

    stripped_text = contents.get_text()
    # 下の正規表現の意味を説明してみよう。
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [26]:
clean_content = strip_html_tags(content)

In [9]:
import sys
sys.path

['/home/input/course-nlp2020/report1',
 '/home/input/course-nlp2020/report1',
 '/opt/facets/facets_overview/python',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/root/.local/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages',
 '/src/bq-helper',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

In [16]:
%cd /home/input/course-nlp2020/wikiextractor/wikiextractor/

/home/input/course-nlp2020/wikiextractor/wikiextractor


In [18]:
!python3 -m WikiExtractor.py ../../report1/Yoshi

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/opt/conda/lib/python3.7/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/home/input/course-nlp2020/wikiextractor/wikiextractor/WikiExtractor.py", line 62, in <module>
    from .extract import Extractor, ignoreTag
ImportError: attempted relative import with no known parent package


In [62]:
clean_content = re.sub("mw-parser-output.*ont-weight:inherit","",clean_content)

In [117]:
# print(clean_content)

In [63]:
pip install -U spacy

     |████████████████████████████████| 9.9 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 26.9 MB/s eta 0:00:01
ERROR: allennlp 0.9.0 has requirement spacy<2.2,>=2.1.0, but you'll have spacy 2.3.2 which is incompatible.
  Attempting uninstall: thinc
    Found existing installation: thinc 7.3.1
    Uninstalling thinc-7.3.1:
      Successfully uninstalled thinc-7.3.1
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.3
    Uninstalling spacy-2.2.3:
      Successfully uninstalled spacy-2.2.3
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [65]:
import spacy
# 英語のtokenizer、tagger、parser、NER、word vectorsをインポート
nlp = spacy.load('en_core_web_sm')

In [90]:
# サンプルテキストに対する固有表現とエンティティタイプの抽出
text = u'My name is Ishio from Japan. Today, talk football!!'
doc = nlp(clean_content)
token = doc[3] # Ishio
result_content = ""
result_array = []
for d in doc:
    if d.is_stop:
        continue
    if d.pos_ == "PROPN":
        result_content = result_content+" "+str(d)
        result_array.append(str(d))
    else:
        result_content = result_content+" "+str(d.lemma_.lower())
        result_array.append(str(d.lemma_.lower()))
        

## 各単語の出現回数を求め、表示する。

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
count2 = CountVectorizer()
bags2 = count2.fit_transform([result_content])

In [115]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:3000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [116]:
import pandas as pd
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 2000)
df = pd.DataFrame( bags2.toarray(), columns=count2.get_feature_names()  )
print(df.T)

                                                 0
01                                               1
02                                               2
03                                               3
04                                               1
05                                               1
06                                               2
07                                               1
08                                               2
09                                              10
10                                               5
100                                              3
104117                                           1
11                                               9
12                                               7
128                                              1
13                                               1
14                                               1
15                                               1
16                             

## lemmatizationした後の単語を、元の出現順序どおりに、半角スペースで区切ってつなぎ、長い一つの文字列にする。

In [106]:
" ".join(result_array)

'\n Yoshi \n Wikipedia , free encyclopedia \n Jump navigation \n jump search \n Nintendo character \n article Nintendo character . use , Yoshi ( disambiguation ) . \n YoshiMario Yoshi characteryoshi hold egg , depict promotional artwork Yoshi Crafted WorldFirst appearancesuper Mario World ( 1990)last appearancesuper Mario 3d - Stars ( 2020)created byshigefumi hino[1]voiced byKazumi Totaka ( video game ; 1997–present)Andrew Sabiston ( Super Mario World)Frank Welker ( Super Mario Bros.)In - universe informationfull namet. Yoshisaur MunchakoopasSpeciesYoshiGenderMaleSignificant otherbirdo \n yoshi[a ] fictional dinosaur[5 ] appear video game publish Nintendo.[6 ] Yoshi debut Super Mario World ( 1990 ) Super Nintendo Entertainment System[7 ] Mario Luigi sidekick . Yoshi later star platform puzzle game , include Super Mario World 2 : Yoshi Island , Yoshi Story , Yoshi Woolly world , Yoshi crafted world . Yoshi appear Mario spin - game , include Mario Party Mario Kart , Mario sport game , Ni